# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="140" /> **IMT2250 - Optimización para  Ciencia de Datos**
**Pontificia Universidad Católica de Chile**<br>
**Semestre 2023-2**<br>
**Profesor:** Alejandro Cataldo <br>
**Estudiantes:** Matías Pineda - Tomás Romero

In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
from math import comb
import warnings
from random import sample
warnings.filterwarnings('ignore')

## Parte Computacional

Considere que, apoyándose en el desarrollo de la Ciencia de Datos y la Optimización avanzada, usted desea conformar un nuevo equipo de fútbol. Para esto, usted ha logrado recopilar información sobre 325 futbolistas. Esta información corresponde a la valoración que se tiene, para cada uno de ellos, de las 4 posiciones en el campo de juego: Arquero, Defensa, Mediocampista y Delantero; y al precio de contratar a ese jugador. Esta valoración está medida en un coeficiente que va desde el 0 al 100, donde el 100 es la mayor valoración. Además, cuanta con la información de qué jugadores no deberían compartir equipo con qué jugadores. Esta información está clasificada como 0 (pueden compartir equipo), 1 (conflicto 1: preferentemente no deberían compartir equipo) y 2 (conflicto 2: no deberían compartir equipo). Toda esta información se encuentra en el archivo Datos.xls, en la hojas Características y Conflictos. Basado en esta información, se le pide conformar un equipo con la mayor valoración posible que cuente con 33 jugadores (no se puede contratar a un jugador en más de una posición), de los cuales 3 son Arqueros, 10 Defensas, 12 son Mediocampistas y 8 son Delanteros, y que se limite por el presupuesto disponible, que es de 22000 pesos. Para esto, usted sabe que cada jugador de la lista puede ser contratado en cualquier de las 4 posiciones, que la valoración total del equipo es la suma de las valoraciones individuales de la posición escogida para cada jugador contratado, y que el costo total de la plantilla de jugadores es la suma de los precios de contratación de todos los jugadores contratados.

(a) (10 puntos) Diseñe un algoritmo, prográmelo y ejecútelo en Python, para seleccionar aleatoriamente jugadores a cada una de las 33 posiciones requeridas sin sobrepasar el presupuesto disponible. Registre la solución en: jugadores seleccionados y su posición, valoración total del equipo, total de conflictos (separados en conflictos 1 y conflictos 2) y listados de casos (jugadores en cada conflicto), y el costo total de la plantilla. Una vez diseñado este algoritmo, ejecútelo 1000 veces, y entregue la solución que tiene menos puntaje en conflictos (sume 1 punto a cada conflicto 1 y 5 puntos a cada conflicto 2). En
caso de que haya más de una solución con el mismo puntaje en conflictos, escoja aquella que tenga mayor valoración total.

In [2]:
caract = pd.read_excel('Datos.xlsx', sheet_name='Características')
caract

,Nombre,Arquero,Defensa,Mediocampista,Delantero,Costo
0,Jugador 1,17,3,74,72,542
1,Jugador 2,14,24,20,78,789
2,Jugador 3,98,69,83,11,523
3,Jugador 4,18,12,28,12,577
4,Jugador 5,36,81,49,52,679
...,...,...,...,...,...,...
320,Jugador 321,92,47,71,6,836
321,Jugador 322,89,37,91,74,931
322,Jugador 323,26,99,58,28,703
323,Jugador 324,94,98,31,93,748


In [3]:
conflict = pd.read_excel('Datos.xlsx', sheet_name='Conflictos', index_col=0)
conflict

,Jugador 1,Jugador 2,Jugador 3,Jugador 4,Jugador 5,Jugador 6,Jugador 7,Jugador 8,Jugador 9,Jugador 10,...,Jugador 316,Jugador 317,Jugador 318,Jugador 319,Jugador 320,Jugador 321,Jugador 322,Jugador 323,Jugador 324,Jugador 325
Jugador 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jugador 2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jugador 3,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jugador 4,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jugador 5,0.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jugador 321,0.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
Jugador 322,0.0,1.0,1.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,...,2.0,0.0,1.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN
Jugador 323,2.0,0.0,0.0,2.0,1.0,1.0,2.0,0.0,1.0,1.0,...,2.0,0.0,1.0,2.0,1.0,0.0,1.0,NaN,NaN,NaN
Jugador 324,0.0,0.0,2.0,0.0,2.0,1.0,1.0,1.0,0.0,2.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN


## Clase Jugadores: 
--- 

In [4]:
class Jugador:
    def __init__(self, j, arquero, defensa, mediocampista, delantero, precio):
        self.j = j
        self.arquero = arquero
        self.defensa = defensa
        self.mediocampista = mediocampista
        self.delantero = delantero
        self.precio = precio
        self.comprado = False
        self.pos = None
        self.val = None

def asignar_pos(jugadores, pos):
    for i in range(len(jugadores)):
        jugadores[i].pos = pos
        jugadores[i].val = jugadores[i].__dict__[pos]
    return jugadores

## Creación del los jugadores:
--- 

In [5]:
jugadores = []
for i in range(len(caract)):
    jugador = caract["Nombre"][i]
    arquero = caract["Arquero"][i]
    defensa = caract["Defensa"][i]
    mediocampista = caract["Mediocampista"][i]
    delantero = caract["Delantero"][i]
    costo = caract["Costo"][i]
    jugadores.append(Jugador(jugador, arquero, defensa, mediocampista, delantero, costo))

## Búsqueda del equipo:
--- 

In [6]:
presupuesto = 22000

puntaje_conflictos_final = np.inf
jugadores_seleccionados = []
conflictos_final = None
val_total_final = None
equipo_final = None
costo_total = None
cant_c1 = None
cant_c2 = None

for _ in range(1000):
    #Conflictos
    c1 = 0
    c2 = 0

    #Crear equipo al azar el cual no sobrepase el presupuesto
    costo = presupuesto + 1
    while costo > presupuesto:
        costo = 0
        equipo = sample(jugadores, 33)
        for j in equipo:
            costo += j.precio

    #Particionamos el equipo
    arqueros = equipo[:3]       ;   arqueros = asignar_pos(arqueros, "arquero")
    defensas = equipo[3:13]     ;   defensas = asignar_pos(defensas, "defensa")
    medios = equipo[13:25]      ;   medios = asignar_pos(medios, "mediocampista")
    delanteros = equipo[25:33]  ;   delanteros = asignar_pos(delanteros, "delantero")

    #Conflictos
    conflictos = []
    for jugador1 in equipo:
        for jugador2 in equipo:
            if conflict[jugador1.j][jugador2.j] == 1:
                c1 += 1
                conflictos.append((jugador1.j, jugador2.j))
            if conflict[jugador1.j][jugador2.j] == 2:
                c2 += 1
                conflictos.append((jugador1.j, jugador2.j))
    puntaje_conflictos = c1 + c2*5
    val_total = sum([j.val for j in equipo])
    #Optimo
    if (puntaje_conflictos < puntaje_conflictos_final) or \
        (puntaje_conflictos == puntaje_conflictos_final and val_total > val_total_final): #Igual puntaje, pero seleccionamos mayor valoracion
        equipo_final = equipo
        puntaje_conflictos_final = c1 + c2*5
        val_total_final = val_total
        cant_c1 = c1
        cant_c2 = c2
        conflictos_final = conflictos
        costo_total = costo

        for jugador in equipo:
            jugadores_seleccionados.append((jugador.j, jugador.pos))


## Resultados:
--- 

In [7]:
print("Equipo:", jugadores_seleccionados)
print("Valor total del equipo:", val_total_final)
print("Cantidad de conflictos totales:", cant_c1 + cant_c2)
print("Cantidad de conflictos tipo 1:", cant_c1)
print("Cantidad de conflictos tipo 2:", cant_c2)
print("Conflictos:", conflictos_final)
print("Puntaje de conflictos:", puntaje_conflictos_final)
print("Costo total del equipo:", costo_total)

Equipo: [('Jugador 265', 'arquero'), ('Jugador 288', 'arquero'), ('Jugador 225', 'arquero'), ('Jugador 102', 'defensa'), ('Jugador 208', 'defensa'), ('Jugador 247', 'defensa'), ('Jugador 255', 'defensa'), ('Jugador 200', 'defensa'), ('Jugador 229', 'defensa'), ('Jugador 305', 'defensa'), ('Jugador 16', 'defensa'), ('Jugador 144', 'defensa'), ('Jugador 248', 'defensa'), ('Jugador 289', 'mediocampista'), ('Jugador 69', 'mediocampista'), ('Jugador 94', 'mediocampista'), ('Jugador 76', 'mediocampista'), ('Jugador 294', 'mediocampista'), ('Jugador 194', 'mediocampista'), ('Jugador 168', 'mediocampista'), ('Jugador 170', 'mediocampista'), ('Jugador 96', 'mediocampista'), ('Jugador 99', 'mediocampista'), ('Jugador 9', 'mediocampista'), ('Jugador 309', 'mediocampista'), ('Jugador 2', 'delantero'), ('Jugador 232', 'delantero'), ('Jugador 4', 'delantero'), ('Jugador 167', 'delantero'), ('Jugador 72', 'delantero'), ('Jugador 35', 'delantero'), ('Jugador 124', 'delantero'), ('Jugador 206', 'delant

In [8]:
# Para Evitar Caídas en el Código, reflejamos la matriz de forma reciproca para los pares de jugadores
aux = conflict
aux = aux.T
aux = aux.fillna(0)

conflict2 = conflict.fillna(0)
conflict = conflict2.add(aux)
for i in range(325):
    conflict[f'Jugador {i + 1}'][f'Jugador {i + 1}'] = -1 # No pueden haber conflicto con uno mismo
    conflict[f'Jugador {i + 1}'] = conflict[f'Jugador {i + 1}'].astype(int)
conflict.head()

,Jugador 1,Jugador 2,Jugador 3,Jugador 4,Jugador 5,Jugador 6,Jugador 7,Jugador 8,Jugador 9,Jugador 10,...,Jugador 316,Jugador 317,Jugador 318,Jugador 319,Jugador 320,Jugador 321,Jugador 322,Jugador 323,Jugador 324,Jugador 325
Jugador 1,-1,2,0,0,0,0,1,1,0,0,...,1,1,0,2,1,0,0,2,0,2
Jugador 2,2,-1,1,0,1,0,0,0,2,1,...,0,2,0,1,2,1,1,0,0,1
Jugador 3,0,1,-1,0,0,1,2,0,0,0,...,0,0,2,1,0,0,1,0,2,1
Jugador 4,0,0,0,-1,2,2,0,0,0,0,...,0,1,0,1,0,1,0,2,0,0
Jugador 5,0,1,0,2,-1,2,2,0,1,1,...,0,1,0,0,1,1,1,1,2,0


(b) (10 puntos) Diseñe un modelo de optimización lineal en variables binarias, prográmelo y resuélvalo en Python/GUROBI, para encontrar la conformación de la plantilla de jugadores (33 posiciones), evitando que no existan conflictos tipo 1 y que no se sobrepasa el presupuesto disponible. Compare esta solución con la obtenida en la parte (a).

In [9]:
P = {0:'Arquero', 1:'Defensa', 2:'Mediocampista', 3:'Delantero'}

## Variables:
$$ x_{i} := 1 \text{ si el jugador } i \text{ juega en el equipo. 0 en otro caso}\hspace{1cm}$$
$$ y_{ip} := 1 \text{ si el jugador } i \text{ juega en la posición } p \text{. 0 en otro caso}\hspace{1cm}$$
## Parámetros:
$$ c_{i} := \text{costo del jugador } i. \hspace{1cm} \in [0, 1000]$$
$$ w_{ij} := \text{tipo del conflicto entre el jugador } i \text{ y el jugador } j. \hspace{1cm} \in \{0, 1, 2\}$$
$$ z_{ip} := \text{valor del jugador } i \text{ en la posición } p. \hspace{1cm} \in [0, 100]$$

In [10]:
model = gp.Model()
model

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-22


<gurobi.Model Continuous instance Unnamed: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>

In [11]:
X = model.addVars(325, vtype=gp.GRB.BINARY)

In [12]:
Y = model.addVars(325, 4, vtype=gp.GRB.BINARY)

### Función Objetivo:
$$ \max \sum_{i = 1}^{325} \sum_{p = 0}^{3} x_{i} \cdot y_{ip} \cdot z_{ip} - \sum_{i = 1}^{324}\sum_{j = i + 1}^{325} x_i \cdot x_j$$

In [13]:
model.setObjective(sum(sum(X[i] * Y[i, p] * caract.iloc[i][P[p]] for p in range(4)) for i in range(325))
                  - sum(X[i]*X[j] if conflict[f'Jugador {i + 1}'][f'Jugador {j + 1}'] == 1 else 0 for i in range(324) for j in range(i + 1, 325)),
                  gp.GRB.MAXIMIZE)

### Restricciones:

El costo total del equipo no supere el presupuesto dado.
$$\sum_{i = 1}^{325} x_{i}\cdot c_{i} \leq 22000$$

In [14]:
model.addConstr(sum(caract.iloc[i]['Costo'] * X[i] for i in range(325)) <= 22000)

<gurobi.Constr *Awaiting Model Update*>

Deben haber 33 jugadores en el equipo.
$$\sum_{i = 1}^{325} x_{i} = 33$$

In [15]:
model.addConstr(sum(X[i] for i in range(325)) == 33)

<gurobi.Constr *Awaiting Model Update*>

Un jugador en el equipo sólo debe contratarse en una sóla posición.
$$ \sum_{p = 0}^{3}y_{ip} = x_i \hspace{1cm} \forall i=1, \dots, 325$$

In [16]:
for i in range(325):
    model.addConstr(sum(Y[(i, p)] for p in range(4)) == X[i])

Deben haber 3 jugadores en la posición de Arquero.
$$\sum_{i = 1}^{325} y_{i0} = 3$$

In [17]:
model.addConstr(sum(Y[i, 0] for i in range(325)) == 3)

<gurobi.Constr *Awaiting Model Update*>

Deben haber 10 jugadores en la posición de Defensa.
$$\sum_{i = 1}^{325} y_{i1} = 10$$

In [18]:
model.addConstr(sum(Y[i, 1] for i in range(325)) == 10)

<gurobi.Constr *Awaiting Model Update*>

Deben haber 12 jugadores en la posición de Mediocampista.
$$\sum_{i = 1}^{325} y_{i2} = 12$$

In [19]:
model.addConstr(sum(Y[i, 2] for i in range(325)) == 12)

<gurobi.Constr *Awaiting Model Update*>

Deben haber 8 jugadores en la posición de Delantero.
$$\sum_{i = 1}^{325}y_{i3} = 8$$

In [20]:
model.addConstr(sum(Y[i, 3] for i in range(325)) == 8)

<gurobi.Constr *Awaiting Model Update*>

In [21]:
model.update()

In [22]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 5 4500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Optimize a model with 331 rows, 1625 columns and 3575 nonzeros
Model fingerprint: 0x88c08cff
Model has 18257 quadratic objective terms
Variable types: 0 continuous, 1625 integer (1625 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+04]
Found heuristic solution: objective 1616.0000000
Presolve time: 0.05s
Presolved: 656 rows, 1950 columns, 22479 nonzeros
Variable types: 0 continuous, 1950 integer (1625 binary)

Root relaxation: objective 7.797820e+03, 418 iterations, 0.03 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

## Ver quienes son parte de la solución
---

In [29]:
values = pd.Series([X[i].x for i in range(325)])
pos = [(i, j) for i in range(325) for j in range(4) if Y[i , j].x == 1]

In [30]:
equipo_guro = caract[values == 1]
for i in range(33):
    for j in range(4):
        if j != pos[i][1]:
            equipo_guro[P[j]][pos[i][0]] = np.nan

In [31]:
equipo_guro

,Nombre,Arquero,Defensa,Mediocampista,Delantero,Costo
10,Jugador 11,NaN,NaN,99.0,NaN,996
15,Jugador 16,NaN,99.0,NaN,NaN,849
19,Jugador 20,NaN,NaN,NaN,99.0,606
21,Jugador 22,NaN,93.0,NaN,NaN,663
26,Jugador 27,NaN,99.0,NaN,NaN,594
36,Jugador 37,NaN,NaN,NaN,95.0,424
49,Jugador 50,NaN,NaN,NaN,99.0,962
68,Jugador 69,NaN,98.0,NaN,NaN,364
70,Jugador 71,NaN,98.0,NaN,NaN,475
76,Jugador 77,NaN,NaN,NaN,100.0,984


### Conflictos de los jugadores:
 

In [26]:
c = {1: 0, 2: 0, 0: 0}
for i in range(32):
    for j in range(i + 1, 33):
        c[conflict[equipo_guro.iloc[i]['Nombre']][equipo_guro.iloc[j]['Nombre']]] += 1
c

{1: 108, 2: 108, 0: 312}

### Valoración Total del Equipo:
 

In [27]:
val_total_guro = 0
for pos in ['Arquero','Defensa','Mediocampista', 'Delantero']:
    val_total_guro += equipo_guro[pos].sum()
val_total_guro

3222.0

### Comparación con los resultados obtenidos en (a):
 

In [28]:
cant_c1, cant_c2, val_total_final

(171, 66, 1459)

Vemos que la solución obtenida por GUROBY obtiene una mayor valoración total considerable, disminuye 63 conflictos tipo 1, pero aumenta 42 conflictos tipo 2.